In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
import numpy as np
device = torch.device("mps")
print(device)

block_size = 8
batch_size = 16
max_iters = 3000
#eval_interval = 2500
learning_rate = 3e-4 # 3e-3, 3e-4, 1e-3, 1e-4
eval_iters = 100
dropout = 0.2
n_embd = 384
n_layer = 8
n_head = 8

mps


#### dataloader

In [14]:
chars = ""
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
    
vocab_size = len(chars)

#### tokenizer

In [15]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

# encoding-decoding tokenizer (character level)
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [16]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

In [17]:
@torch.no_grad() # decorator that doesnt allow gradient calc
def estimate_loss():
    out = {}
    model.eval() # dropout is turned off at evaluation
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5  # (B, T, hs) -> (B, T, T)

        # Dynamically create the lower triangular mask based on the current sequence length
        tril_mask = torch.tril(torch.ones(T, T, device=x.device))
        wei = wei.masked_fill(tril_mask == 0, float('-inf'))  # (B, T, T)

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)  # (B, T, hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

    
#     def forward(self, x):
#         # input of size (batch, time-step, channel)
#         # output of size (batch, time-step, head size)
#         B,T,C = x.shape
#         k = self.key(x)
#         q = self.query(x)
#         # compute attention scores ("affinities")
#         wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,hs) -> (B, T, T)
#         wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
#         wei = F.softmax(wei, dim=-1) #(B,T,T)
#         wei = self.dropout(wei)
#         # perform the weighted aggregation of the value
#         v = self.value(x) # (B,T,hs)
#         out = wei @ v # (B,T,T) @ (B,T,hs) -> (B,T,hs)
#         return out


class MultiHeadAttention(nn.Module):
    """ multiple head of self.attention is parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # (B, T, Feature) --> (B, T, [h1,h1,h1,h1,h2,h2,h2,h2,h3,h3,h3,h3])
        out = torch.cat([h(x) for h in self.heads], dim=-1) 
        out = self.dropout(self.proj(out))
        return out 

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embd) # final norm layer
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                
        
    def forward(self, index, targets=None):
        B, T = index.shape
        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(index) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [19]:
model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [20]:
# create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    # sample a batch of data
    xb, yb = get_batch("train")
    
    # evaluate the loss
    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step: 0, train loss: 4.4426, val loss: 4.4430
step: 100, train loss: 2.5108, val loss: 2.5565
step: 200, train loss: 2.3802, val loss: 2.4197
step: 300, train loss: 2.2892, val loss: 2.3394
step: 400, train loss: 2.2377, val loss: 2.3093
step: 500, train loss: 2.2054, val loss: 2.2537
step: 600, train loss: 2.1299, val loss: 2.2460
step: 700, train loss: 2.1156, val loss: 2.1703
step: 800, train loss: 2.0766, val loss: 2.1878
step: 900, train loss: 2.0629, val loss: 2.1545
step: 1000, train loss: 2.0497, val loss: 2.1005
step: 1100, train loss: 2.0016, val loss: 2.0775
step: 1200, train loss: 2.0073, val loss: 2.0563
step: 1300, train loss: 1.9739, val loss: 2.0493
step: 1400, train loss: 1.9522, val loss: 2.0317
step: 1500, train loss: 1.9195, val loss: 1.9786
step: 1600, train loss: 1.9478, val loss: 2.0117
step: 1700, train loss: 1.9316, val loss: 2.0015
step: 1800, train loss: 1.9114, val loss: 2.0037
step: 1900, train loss: 1.9165, val loss: 2.0368
step: 2000, train loss: 1.8870, 

In [21]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=100)[0].tolist())
print(generated_chars)


with whith oullernonothily e ay ple
imavind d fumboup utt hit t bory ne
shinoup s d h ole, d mar
rin


In [24]:
prompt = 'Hey girl'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hey girlefoosesm?" finche e t ed slomed lasleche t," w, me wan tid inifuse fley D"Cheshonoup ousthesinos an 
